In [ ]:
import pandas as pd
from os import getcwd
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
from UliPlot.XLSX import auto_adjust_xlsx_column_width

: 

In [ ]:
strain_query = '''
SELECT  [StrainGauge].[gid]
      ,[gpn]
      ,[ver] as 'PCBver'
      ,[toolno]
      ,[station]
      ,[Position]
      ,[TestRecord].judge
      ,[TestRecord].stress
      ,[TestRecord].WQ211
      ,[memo]
      ,[org]
      ,[createdata]
  FROM [SMTvip].[dbo].[StrainGauge]

  Left JOIN [SMTvip].[dbo].[TestRecord] 
  on [StrainGauge].[gid] = [TestRecord].[gid]
  where gpn = '105-04263-20'
  ORDER by profdate DESC
'''

connection_string = "DRIVER={SQL Server};SERVER=T1-PE-SUPPORT;DATABASE=PR_WEB2;UID=pimes2;PWD=LhZEv9JtGsZNX"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
raw = pd.read_sql(strain_query, engine)

: 

In [ ]:
def pivot_raw(raw):
    return(raw
    .assign(judge = raw.judge.map({'OK':1, 'NG':0}).astype('int8'))
    .sort_values('createdata', ascending=False)
    .drop_duplicates(subset=['PCBver','station','Position','memo'])
    .groupby(['station','memo','PCBver','createdata','Position'])
    ['judge']
    .sum()
    .unstack('Position')
    .sort_index(level=['station','createdata'],ascending=False)
    .style.applymap(lambda val : "background-color: red" if val == 0 else "")
    )
pivot_raw(raw)

: 